In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchvision.transforms as tt
from torch.utils.data import DataLoader, ConcatDataset
import torchvision

stats = ((0.2860,), (0.3530,))

# data from augmentation ablation here
basic_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])
train_fms = tt.Compose([tt.RandomCrop(28, padding=4, padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(p=0.5), 
                         tt.RandomVerticalFlip(p=0.5),
                        tt.ToTensor(), 
                        tt.Normalize(*stats,inplace=True)])

batch_size = 1024

train_normal = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=basic_tfms, download=True)
train_ds = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=train_fms)

train_dataset = ConcatDataset([train_ds, train_normal])
train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=3, pin_memory=True)

val_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=basic_tfms)
valid_dl = DataLoader(val_dataset, batch_size*2, num_workers=3, pin_memory=True)

In [3]:
import torch

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    # elif torch.backends.mps.is_available():
    #     return torch.device("mps")
    else:
        return torch.device("cpu")
    
def clear_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()
    # else:
    #     return torch.device("cpu")
    # appers there's nothing to do here
        
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
    
device = get_default_device()
print(f"running on {device}")

running on cuda


In [4]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = self.conv_block(in_channels, 64)
        self.conv2 = self.conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(self.conv_block(128, 128), self.conv_block(128, 128))
        
        self.conv3 = self.conv_block(128, 256, pool=True)
        self.conv4 = self.conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(self.conv_block(512, 512), self.conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(3), 
                                        nn.Flatten(), 
                                        nn.Linear(512, num_classes))

    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  
        loss = F.cross_entropy(out, labels) 
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

    def conv_block(self, in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                nn.BatchNorm2d(out_channels), 
                nn.ReLU(inplace=True)]
        if pool: 
            layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [6]:
model = to_device(ResNet9(1, 10), device)

In [7]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader),
                                                pct_start=0.3)
    
    for epoch in range(epochs):
        # Training Phase 
        #print(f'Allocated: {torch.cuda.memory_allocated() / 1024 ** 2} MB')
        #print(f'Cached: {torch.cuda.memory_reserved() / 1024 ** 2} MB')
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            #print(f'Allocated: {torch.cuda.memory_allocated() / 1024 ** 2} MB')
            #print(f'Cached: {torch.cuda.memory_reserved() / 1024 ** 2} MB')
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [8]:
epochs = 8
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [9]:
# testing epochs
history_map = {}
index = 0
history = []
for i in range(8,21):
    history = []
    model = to_device(ResNet9(1, 10), device)
    history += fit_one_cycle(i, max_lr, model, train_dl, valid_dl, 
                                 grad_clip=grad_clip, 
                                 weight_decay=weight_decay, 
                                 opt_func=opt_func)
    history_map[index] = history
    index += 1

Epoch [0], last_lr: 0.00393, train_loss: 0.5604, val_loss: 0.5107, val_acc: 0.8248
Epoch [1], last_lr: 0.00935, train_loss: 0.4159, val_loss: 0.3119, val_acc: 0.8913
Epoch [2], last_lr: 0.00972, train_loss: 0.3124, val_loss: 0.4043, val_acc: 0.8673
Epoch [3], last_lr: 0.00812, train_loss: 0.2552, val_loss: 0.2743, val_acc: 0.9068
Epoch [4], last_lr: 0.00556, train_loss: 0.2127, val_loss: 0.3023, val_acc: 0.8977
Epoch [5], last_lr: 0.00283, train_loss: 0.1742, val_loss: 0.1842, val_acc: 0.9336
Epoch [6], last_lr: 0.00077, train_loss: 0.1346, val_loss: 0.1597, val_acc: 0.9429
Epoch [7], last_lr: 0.00000, train_loss: 0.1077, val_loss: 0.1530, val_acc: 0.9462
Epoch [0], last_lr: 0.00327, train_loss: 0.5480, val_loss: 0.3781, val_acc: 0.8650
Epoch [1], last_lr: 0.00848, train_loss: 0.3940, val_loss: 0.4423, val_acc: 0.8571
Epoch [2], last_lr: 0.00994, train_loss: 0.3467, val_loss: 0.3338, val_acc: 0.8804
Epoch [3], last_lr: 0.00899, train_loss: 0.2629, val_loss: 0.2141, val_acc: 0.9204
Epoc

In [10]:
def find_best_transformation(names):
    final_val_acc = [history_map[i][-1]['val_acc'] for i in range(len(history_map))]
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Accuracy': final_val_acc
    })
    # Ensure 'Final Validation Accuracy' is treated as numeric
    df['Final Validation Accuracy'] = pd.to_numeric(df['Final Validation Accuracy'], errors='coerce')

    # Find the row with the highest accuracy
    best_row = df.loc[df['Final Validation Accuracy'].idxmax()]
    print(f" best: {best_row['Transformation']} acc: {best_row['Final Validation Accuracy']}")

    # Return the transformation and the best accuracy
    return best_row['Transformation']

In [11]:
import pandas as pd

def compare_final_val_losses(names):
    # Create a list of final validation losses
    final_val_losses = [history_map[i][-1]['val_loss'] for i in range(len(history_map))]
    final_val_acc = [history_map[i][-1]['val_acc'] for i in range(len(history_map))]

    # Create a pandas DataFrame and display it
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Loss': final_val_losses
    })
    print(df)
    print("*"*10)
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Accuracy': final_val_acc
    })
    print(df)
    
compare_final_val_losses(range(8,21))

    Transformation  Final Validation Loss
0                8               0.153036
1                9               0.155030
2               10               0.152813
3               11               0.155522
4               12               0.157033
5               13               0.155769
6               14               0.159400
7               15               0.157522
8               16               0.157775
9               17               0.153617
10              18               0.161962
11              19               0.162707
12              20               0.161572
**********
    Transformation  Final Validation Accuracy
0                8                   0.946204
1                9                   0.948118
2               10                   0.949113
3               11                   0.946815
4               12                   0.948391
5               13                   0.947317
6               14                   0.949771
7               15               

In [14]:
# best lr

import numpy as np
best_epoch = 16
history_map = {}
index = 0

for i in np.arange(0.001, 0.011, 0.001):
    history = []
    model = to_device(ResNet9(1, 10), device)
    history += fit_one_cycle(best_epoch, i, model, train_dl, valid_dl, 
                                 grad_clip=grad_clip, 
                                 weight_decay=weight_decay, 
                                 opt_func=opt_func)
    history_map[index] = history
    index += 1

compare_final_val_losses(np.arange(0.001, 0.011, 0.001))

Epoch [0], last_lr: 0.00014, train_loss: 0.7385, val_loss: 0.3612, val_acc: 0.8641
Epoch [1], last_lr: 0.00039, train_loss: 0.3774, val_loss: 0.2937, val_acc: 0.8972
Epoch [2], last_lr: 0.00070, train_loss: 0.3140, val_loss: 0.3416, val_acc: 0.8793
Epoch [3], last_lr: 0.00094, train_loss: 0.2775, val_loss: 0.2681, val_acc: 0.9056
Epoch [4], last_lr: 0.00100, train_loss: 0.2411, val_loss: 0.2715, val_acc: 0.9039
Epoch [5], last_lr: 0.00097, train_loss: 0.2111, val_loss: 0.2457, val_acc: 0.9105
Epoch [6], last_lr: 0.00091, train_loss: 0.1879, val_loss: 0.1863, val_acc: 0.9305
Epoch [7], last_lr: 0.00081, train_loss: 0.1653, val_loss: 0.2193, val_acc: 0.9233
Epoch [8], last_lr: 0.00069, train_loss: 0.1513, val_loss: 0.2204, val_acc: 0.9230
Epoch [9], last_lr: 0.00056, train_loss: 0.1320, val_loss: 0.2485, val_acc: 0.9195
Epoch [10], last_lr: 0.00042, train_loss: 0.1182, val_loss: 0.1818, val_acc: 0.9427
Epoch [11], last_lr: 0.00028, train_loss: 0.1030, val_loss: 0.1755, val_acc: 0.9434
Ep

In [15]:
# best weight decay

import numpy as np
best_lr = find_best_transformation(np.arange(0.001, 0.011, 0.001))
history_map = {}
index = 0

for i in np.arange(1e-4, 6e-4, 1e-4):
    history = []
    model = to_device(ResNet9(1, 10), device)
    history += fit_one_cycle(best_epoch, best_lr, model, train_dl, valid_dl, 
                                 grad_clip=grad_clip, 
                                 weight_decay=i, 
                                 opt_func=opt_func)
    history_map[index] = history
    index += 1

 best: 0.007 acc: 0.9527066946029663
Epoch [0], last_lr: 0.00097, train_loss: 0.5355, val_loss: 0.3262, val_acc: 0.8862
Epoch [1], last_lr: 0.00276, train_loss: 0.3327, val_loss: 0.6080, val_acc: 0.7970
Epoch [2], last_lr: 0.00492, train_loss: 0.3061, val_loss: 0.3920, val_acc: 0.8719
Epoch [3], last_lr: 0.00655, train_loss: 0.2791, val_loss: 0.3198, val_acc: 0.8900
Epoch [4], last_lr: 0.00699, train_loss: 0.2534, val_loss: 0.3363, val_acc: 0.8797
Epoch [5], last_lr: 0.00680, train_loss: 0.2214, val_loss: 0.2386, val_acc: 0.9162
Epoch [6], last_lr: 0.00635, train_loss: 0.1994, val_loss: 0.2960, val_acc: 0.8974
Epoch [7], last_lr: 0.00568, train_loss: 0.1920, val_loss: 0.2008, val_acc: 0.9254
Epoch [8], last_lr: 0.00484, train_loss: 0.1670, val_loss: 0.1820, val_acc: 0.9370
Epoch [9], last_lr: 0.00389, train_loss: 0.1506, val_loss: 0.2231, val_acc: 0.9243
Epoch [10], last_lr: 0.00291, train_loss: 0.1319, val_loss: 0.1920, val_acc: 0.9347
Epoch [11], last_lr: 0.00198, train_loss: 0.1188,

In [20]:
compare_final_val_losses(np.arange(1e-4, 6e-4, 1e-4))
best_weight_decay = find_best_transformation(np.arange(1e-4, 6e-4, 1e-4))


In [22]:
# best optimizer
history_map = {}
best_weight_decay = 1e-4

opt_func_list = ["Adam", "AdamW", "SGD"]

history = []
model = to_device(ResNet9(1, 10), device)
history += fit_one_cycle(best_epoch, best_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=best_weight_decay, 
                             opt_func=torch.optim.Adam)
history_map[0] = history

history = []
model = to_device(ResNet9(1, 10), device)
history += fit_one_cycle(best_epoch, best_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=best_weight_decay, 
                             opt_func=torch.optim.AdamW)
history_map[1] = history

history = []
model = to_device(ResNet9(1, 10), device)
history += fit_one_cycle(best_epoch, best_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=best_weight_decay, 
                             opt_func=torch.optim.SGD)
history_map[2] = history

compare_final_val_losses(opt_func_list)

Epoch [0], last_lr: 0.00097, train_loss: 0.5353, val_loss: 0.4381, val_acc: 0.8431
Epoch [1], last_lr: 0.00276, train_loss: 0.3424, val_loss: 0.3907, val_acc: 0.8555
Epoch [2], last_lr: 0.00492, train_loss: 0.3115, val_loss: 0.8307, val_acc: 0.7607
Epoch [3], last_lr: 0.00655, train_loss: 0.2813, val_loss: 0.3088, val_acc: 0.8870
Epoch [4], last_lr: 0.00699, train_loss: 0.2517, val_loss: 0.2469, val_acc: 0.9115
Epoch [5], last_lr: 0.00680, train_loss: 0.2232, val_loss: 0.2203, val_acc: 0.9198
Epoch [6], last_lr: 0.00635, train_loss: 0.2054, val_loss: 0.2334, val_acc: 0.9167
Epoch [7], last_lr: 0.00568, train_loss: 0.1836, val_loss: 0.2436, val_acc: 0.9125
Epoch [8], last_lr: 0.00484, train_loss: 0.1687, val_loss: 0.2235, val_acc: 0.9248
Epoch [9], last_lr: 0.00389, train_loss: 0.1575, val_loss: 0.2424, val_acc: 0.9166
Epoch [10], last_lr: 0.00291, train_loss: 0.1347, val_loss: 0.1841, val_acc: 0.9393
Epoch [11], last_lr: 0.00198, train_loss: 0.1160, val_loss: 0.2005, val_acc: 0.9363
Ep